In [45]:
import mido
import string
import numpy as np
#mid = mido.MidiFile('.mid', clip=True)
mid = mido.MidiFile('test.mid', clip=True)
mid.tracks

[<midi track 'Piano\x00' 63 messages>]

In [64]:
def parse_MIDI(midi_file, user_tempo):
    # ticks_per_quarter = <PPQ from the header>
    # µs_per_quarter = <Tempo in latest Set Tempo event>
    # µs_per_tick = µs_per_quarter / ticks_per_quarter
    # seconds_per_tick = µs_per_tick / 1.000.000
    # seconds = ticks * seconds_per_tick
    clocks_per_click = 0
    midi_info = []
    note_value = 0
    note_velocity = 0
    note_dur = 0
    sec_per_click = 0


    mid = mido.MidiFile(midi_file, clip=True)
    for i in range(0, 2):
        for m in mid.tracks[i][:]:
            print(m)
            m = str(m)
            if 'clocks_per_click' in m and clocks_per_click == 0:
                # midi.append(m)
                temp = m.partition("clocks_per_click=")[2]
                clocks_per_click = temp.partition(" ")[0]
                sec_per_click = convert_clocks_per_click(int(clocks_per_click), user_tempo)
            if 'note_on' in m:
                temp = m.partition("note=")[2]
                note_value = temp.partition(" ")[0]
                #convert to frequency
                note_value = convert_frequency(int(note_value))
                temp = m.partition("velocity=")[2]
                note_velocity = temp.partition(" ")[0]
                #convert to 0-1 amplitude
                note_velocity = convert_velocity(int(note_velocity))
            if 'note_off' in m:
                note_dur = m.partition("time=")[2]
                #convert to seconds
                note_dur = sec_per_click * int(note_dur)
                midi_info.append((note_value, note_velocity, note_dur))
                # print(midi_info)
    return midi_info

In [71]:
note_list = parse_MIDI('scale.mid', 60)

<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
<meta message key_signature key='C' time=0>
<meta message set_tempo tempo=250000 time=0>
<meta message copyright text='Copyright © ' time=0>
<meta message time_signature numerator=10 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=38400>
<meta message time_signature numerator=6 denominator=4 clocks_per_click=72 notated_32nd_notes_per_beat=8 time=9600>
<meta message end_of_track time=1>
program_change channel=0 program=0 time=0
control_change channel=0 control=121 value=0 time=0
control_change channel=0 control=64 value=0 time=0
control_change channel=0 control=91 value=48 time=0
control_change channel=0 control=10 value=51 time=0
control_change channel=0 control=7 value=100 time=0
<meta message track_name name='Piano' time=0>
pitchwheel channel=0 pitch=0 time=0
<meta message lyrics text='1/1 ' time=0>
note_on channel=0 note=60 velocity=76 time=0
note_off

In [72]:
def genSine(frequency, duration, amplitude = 1, sampleRate = 48000, phase = 0):
    
    #creates a sine wave
    
    import numpy as np
    
    time = np.arange(0, duration, 1/sampleRate)
    return amplitude * np.sin((2*np.pi * frequency * time) + phase)

In [85]:
# play_list = []
# for i in note_list:
#     play_list = genSine(i[0], i[2], i[1])
#     play_list.append(sine)
# play_list
play_list

array([ 0.        ,  0.02049009,  0.04095614, ..., -0.30317496,
       -0.3206631 , -0.33777519])

In [87]:
from IPython.display import Audio
Audio(play_list, rate=48000)

In [67]:
def convert_clocks_per_click(clocks_per_click, user_tempo):
    sec_per_click = .6 / (user_tempo * clocks_per_click)
    return sec_per_click

In [68]:
convert_clocks_per_click(36, 60)

0.0002777777777777778

In [39]:
def convert_velocity(midi_vel):
    velocity = midi_vel / 127.0
    return velocity

In [40]:
convert_velocity(120)

0.9448818897637795

In [41]:
def convert_frequency(midi_val):
    reference = 440
    frequency = (reference / 32) * (2 ** ((midi_val - 9) / 12))
    return frequency

In [42]:
convert_frequency(60)

261.6255653005986